# OK Dokhae 뉴스 기사 데이터 품질 리포트 (Ver.2)

이 노트북은 **뉴스 기사 데이터만** 전처리한 `processed_samples_ver2.json`의 품질을 분석합니다.

## 데이터 출처
- `samples2.json`, `sample3.json` (뉴스 기사)
- `TeL_*.json`, `TeS_*.json` (추가 뉴스 데이터)
- ⚠️ `samples.json` (KLUE/SQuAD)은 **제외**되었습니다.

In [ ]:
# 필요 라이브러리 임포트
import json
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from pathlib import Path
import platform

# 한글 폰트 설정
def set_korean_font():
    system_name = platform.system()
    if system_name == 'Darwin':  # Mac
        plt.rc('font', family='AppleGothic')
    elif system_name == 'Windows':  # Windows
        plt.rc('font', family='Malgun Gothic')
    else:  # Linux (Colab 등)
        plt.rc('font', family='NanumGothic')
    plt.rc('axes', unicode_minus=False)

set_korean_font()
print("한글 폰트 설정 완료")

In [ ]:
# 뉴스 기사 전용 데이터 로드
DATA_PATH = "../data/processed_samples_ver2.json"

try:
    with open(DATA_PATH, "r", encoding="utf-8") as f:
        data = json.load(f)
    print(f"✅ 데이터 로드 성공: {len(data)}개 문서 (뉴스 기사 전용)")
except FileNotFoundError:
    print(f"❌ 파일을 찾을 수 없습니다: {DATA_PATH}")
    print("터미널에서 아래 명령어를 먼저 실행해주세요:")
    print("python -m backend.load_samples --exclude samples.json --output data/processed_samples_ver2.json")

## 1. 전반적 통계

In [ ]:
total_docs = len(data)
total_original_sents = sum(doc['preprocessing_stats']['original_count'] for doc in data)
total_kept_sents = sum(doc['preprocessing_stats']['kept_count'] for doc in data)
total_removed = sum(doc['preprocessing_stats']['removed_count'] for doc in data)
removal_rate = (total_removed / total_original_sents * 100) if total_original_sents > 0 else 0

print("=" * 50)
print("📰 뉴스 기사 데이터셋 요약 (Ver.2)")
print("=" * 50)
print(f"• 총 문서 수: {total_docs}개")
print(f"• 원본 문장 수: {total_original_sents}개")
print(f"• 전처리 후 문장 수: {total_kept_sents}개")
print(f"• 제거된 문장 수: {total_removed}개")
print(f"• 전체 제거율: {removal_rate:.1f}%")

## 2. 시각화 분석

In [ ]:
# 2.1 전처리 전/후 문장 수 비교
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 왼쪽: 바 차트
categories = ['원본 문장', '전처리 후']
values = [total_original_sents, total_kept_sents]
colors = ['#e74c3c', '#27ae60']

bars = axes[0].bar(categories, values, color=colors, width=0.5)
for bar in bars:
    height = bar.get_height()
    axes[0].text(bar.get_x() + bar.get_width()/2., height, 
                 f'{int(height)}개', ha='center', va='bottom', fontsize=12, fontweight='bold')

axes[0].set_title('전처리 전/후 문장 수 비교', fontsize=14)
axes[0].set_ylabel('문장 수')
axes[0].spines['top'].set_visible(False)
axes[0].spines['right'].set_visible(False)

# 오른쪽: 파이 차트 (유지 vs 제거)
pie_labels = ['유지', '제거']
pie_sizes = [total_kept_sents, total_removed]
pie_colors = ['#27ae60', '#e74c3c']
explode = (0.05, 0)

axes[1].pie(pie_sizes, explode=explode, labels=pie_labels, colors=pie_colors,
            autopct='%1.1f%%', startangle=90, pctdistance=0.6)
axes[1].set_title('문장 유지/제거 비율', fontsize=14)

plt.tight_layout()
plt.show()

In [ ]:
# 2.2 질문 가능성 점수 분포
scores = [doc['preprocessing_stats']['avg_question_potential'] 
          for doc in data 
          if doc['preprocessing_stats']['kept_count'] > 0]

plt.figure(figsize=(10, 5))
sns.histplot(scores, kde=True, color='#3498db', bins=15)

plt.axvline(x=0.35, color='red', linestyle='--', linewidth=2, label='기본 임계값 (0.35)')
plt.title('문서별 질문 가능성 점수 분포 (뉴스 기사)', fontsize=14)
plt.xlabel('평균 질문 가능성 점수')
plt.ylabel('문서 수')
plt.legend()
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.show()

print(f"평균 점수: {np.mean(scores):.3f}")
print(f"최소 점수: {np.min(scores):.3f}")
print(f"최대 점수: {np.max(scores):.3f}")

In [ ]:
# 2.3 출처별 문서 분포
source_counts = {}
for doc in data:
    source = doc.get('source', 'Unknown')
    source_counts[source] = source_counts.get(source, 0) + 1

plt.figure(figsize=(10, 6))
sources = list(source_counts.keys())
counts = list(source_counts.values())

bars = plt.barh(sources, counts, color=sns.color_palette('viridis', len(sources)))
plt.xlabel('문서 수')
plt.title('뉴스 출처별 문서 분포', fontsize=14)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

for bar, count in zip(bars, counts):
    plt.text(bar.get_width() + 0.1, bar.get_y() + bar.get_height()/2, 
             f'{count}개', va='center', fontsize=10)

plt.tight_layout()
plt.show()

## 3. 샘플 문장 확인

In [ ]:
print("=" * 60)
print("📝 고품질 뉴스 문장 예시 (질문 유도 가능)")
print("=" * 60)

count = 0
for doc in sorted(data, key=lambda x: x['preprocessing_stats']['avg_question_potential'], reverse=True):
    sentences = doc.get('processed_sentences', [])
    avg_score = doc['preprocessing_stats'].get('avg_question_potential', 0)
    source = doc.get('source', 'Unknown')
    
    if sentences:
        print(f"\n[{source}] (점수: {avg_score:.2f})")
        print("-" * 50)
        for i, sent in enumerate(sentences[:2]):
            print(f"  {i+1}. {sent[:100]}..." if len(sent) > 100 else f"  {i+1}. {sent}")
        count += 1
    if count >= 5:
        break

## 4. 결론

뉴스 기사 데이터는 KLUE/SQuAD 데이터와 달리 **논리적 흐름**이 명확하고 **인과관계**가 풍부합니다.
질문 유도 학습에 더 적합한 데이터셋으로 판단됩니다.